In [2]:
%pip install folium --user


   -------------------- ------------------- 1/2 [folium]
   -------------------- ------------------- 1/2 [folium]
   -------------------- ------------------- 1/2 [folium]
   -------------------- ------------------- 1/2 [folium]
   -------------------- ------------------- 1/2 [folium]
   -------------------- ------------------- 1/2 [folium]
   ---------------------------------------- 2/2 [folium]

Note: you may need to restart the kernel to use updated packages.


In [3]:
import pandas as pd
import folium
from folium.plugins import MarkerCluster,MousePosition
from folium.features import DivIcon

In [4]:
url = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv'
spacex_df = pd.read_csv(url)
spacex_df.head()

,Flight Number,Date,Time (UTC),Booster Version,Launch Site,Payload,Payload Mass (kg),Orbit,Customer,Landing Outcome,class,Lat,Long
0,1,2010-06-04,18:45:00,F9 v1.0 B0003,CCAFS LC-40,Dragon Spacecraft Qualification Unit,0.0,LEO,SpaceX,Failure (parachute),0,28.562302,-80.577356
1,2,2010-12-08,15:43:00,F9 v1.0 B0004,CCAFS LC-40,"Dragon demo flight C1, two CubeSats, barrel o...",0.0,LEO (ISS),NASA (COTS) NRO,Failure (parachute),0,28.562302,-80.577356
2,3,2012-05-22,7:44:00,F9 v1.0 B0005,CCAFS LC-40,Dragon demo flight C2+,525.0,LEO (ISS),NASA (COTS),No attempt,0,28.562302,-80.577356
3,4,2012-10-08,0:35:00,F9 v1.0 B0006,CCAFS LC-40,SpaceX CRS-1,500.0,LEO (ISS),NASA (CRS),No attempt,0,28.562302,-80.577356
4,5,2013-03-01,15:10:00,F9 v1.0 B0007,CCAFS LC-40,SpaceX CRS-2,677.0,LEO (ISS),NASA (CRS),No attempt,0,28.562302,-80.577356


In [5]:
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]
launch_sites_df

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


In [15]:
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)

In [16]:
# Create a blue circle at NASA Johnson Space Center's coordinate with a popup label showing its name
circle = folium.Circle(nasa_coordinate, radius=1000, color='#d35400', fill=True).add_child(folium.Popup('NASA Johnson Space Center'))
# Create a blue circle at NASA Johnson Space Center's coordinate with a icon showing its name
marker = folium.map.Marker(
    nasa_coordinate,
    # Create an icon as a text label
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'NASA JSC',
        )
    )
site_map.add_child(circle)
#site_map.add_child(marker)

In [21]:
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)
for site,lat,long,launch in zip(spacex_df['Launch Site'],spacex_df['Lat'],spacex_df['Long'],spacex_df['class']):
    coordinates = [lat,long]
    # Create a blue circle at NASA Johnson Space Center's coordinate with a popup label showing its name
    if launch == 1:
        circle = folium.Circle(coordinates, radius=100, color="#142FC6", fill=True).add_child(folium.Popup(f'Successful Launch at {site}'))
    else :
        circle = folium.Circle(coordinates, radius=100, color="#C61D14", fill=True).add_child(folium.Popup(f'Failed Launch at {site}'))
    # Create a blue circle at NASA Johnson Space Center's coordinate with a icon showing its name
    marker = folium.map.Marker(
        coordinates,
        icon=DivIcon(
            icon_size=(20,20),
            icon_anchor=(0,0),
            html='<div style="font-size: 6; color:#d35400;"><b>%s</b></div>' % site,
            )
        )
    site_map.add_child(circle)
    site_map.add_child(marker)
site_map

In [22]:
spacex_df.tail(10)

,Launch Site,Lat,Long,class
46,KSC LC-39A,28.573255,-80.646895,1
47,KSC LC-39A,28.573255,-80.646895,1
48,KSC LC-39A,28.573255,-80.646895,1
49,CCAFS SLC-40,28.563197,-80.576820,1
50,CCAFS SLC-40,28.563197,-80.576820,1
51,CCAFS SLC-40,28.563197,-80.576820,0
52,CCAFS SLC-40,28.563197,-80.576820,0
53,CCAFS SLC-40,28.563197,-80.576820,0
54,CCAFS SLC-40,28.563197,-80.576820,1
55,CCAFS SLC-40,28.563197,-80.576820,0


In [24]:
marker_cluster = MarkerCluster()

In [25]:
spacex_df['marker_color'] = spacex_df['class'].apply(lambda x: 'green' if x==1 else 'red')

In [31]:
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)

# Draw circles and text labels
for site, lat, long, launch in zip(spacex_df['Launch Site'], spacex_df['Lat'], spacex_df['Long'], spacex_df['class']):
    coordinates = [lat, long]
    if launch == 1:
        circle = folium.Circle(coordinates, radius=100, color="#142FC6", fill=True).add_child(
            folium.Popup(f'Successful Launch at {site}')
        )
    else:
        circle = folium.Circle(coordinates, radius=100, color="#C61D14", fill=True).add_child(
            folium.Popup(f'Failed Launch at {site}')
        )
    marker = folium.Marker(
        coordinates,
        icon=DivIcon(
            icon_size=(20, 20),
            icon_anchor=(0, 0),
            html=f'<div style="font-size: 6; color:#d35400;"><b>{site}</b></div>'
        )
    )
    site_map.add_child(circle)
    site_map.add_child(marker)

# Now add the marker cluster
marker_cluster = MarkerCluster().add_to(site_map)

# Add markers to the cluster
for index, record in spacex_df.iterrows():
    popup_text = f"{record['Launch Site']} - {'Success' if record['class'] == 1 else 'Failure'}"
    marker = folium.Marker(
        location=[record['Lat'], record['Long']],
        popup=popup_text,
        icon=folium.Icon(color='white', icon_color=record['marker_color'], icon='info-sign')
    )
    marker_cluster.add_child(marker)

# LAST LINE: show the map
site_map


In [32]:
# Add Mouse Position to get the coordinate (Lat, Long) for a mouse over on the map
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)

site_map.add_child(mouse_position)
site_map

In [33]:
from math import sin, cos, sqrt, atan2, radians

def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance